In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from solar import REF_SOLAR_DATA
from batteryopt import process_pge_meterdata, merge_solar_and_load_data, build_tariff, run_optimization

%load_ext autoreload
%autoreload 2

In [ ]:
solar_size_kw = 1.0
batt_size_kwh = 13.5
csv_file = 'data/pge-e78ff14c-c8c0-11ec-8cc7-0200170a3297-DailyUsageData/pge_electric_usage_interval_data_Service 1_1_2024-02-01_to_2025-01-31.csv'

elec_usage = process_pge_meterdata(csv_file)
site_data = merge_solar_and_load_data(elec_usage, solar_size_kw * REF_SOLAR_DATA)
tariff = build_tariff(site_data.index)
battery_dispatch = run_optimization(site_data, tariff, batt_e_max=batt_size_kwh)
all_input = pd.concat([site_data, tariff, battery_dispatch], axis=1)

final_week = all_input.loc[all_input.index[-1] - pd.DateOffset(days=7):]

fig, ax = plt.subplots()
final_week.plot(ax=ax)

In [ ]:
tmp_output = all_input[['load', 'solar', 'px_buy', 'px_sell']]
tmp_output.to_csv('data/tmp_out_full.csv', index=False)
tmp_output.loc[all_input.index[-1] - pd.DateOffset(days=2):].to_csv('data/tmp_out_noindex.csv', index=False)